In [ ]:
import datetime
import os
import pandas as pd

In [ ]:
BASE_PATH = "data/nijmegen/"

In [ ]:
def format_seconds(seconds):
    duration = datetime.timedelta(seconds=seconds)

    # Extract hours, minutes, and seconds from the timedelta object
    hours = duration.seconds // 3600
    minutes = (duration.seconds % 3600) // 60
    seconds = duration.seconds % 60

    # Format the time into HH:MM:SS format
    formatted_time = "{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds)

    return formatted_time

In [ ]:
for meeting_type in os.listdir(BASE_PATH):
    if meeting_type.startswith("."):
        continue
    for year in os.listdir(f"{BASE_PATH}/{meeting_type}"):
        if year.startswith("."):
            continue
        if not os.path.isdir(f"{BASE_PATH}/{meeting_type}/{year}/diorizations"):
            continue

        for dior in os.listdir(f"{BASE_PATH}/{meeting_type}/{year}/diorizations"):
            if dior.startswith("."):
                continue

            start_times = []
            durations = []
            speaker_ids = []
            if not os.path.isdir(f"{BASE_PATH}/{meeting_type}/{year}/sheets"):
                os.mkdir(f"{BASE_PATH}/{meeting_type}/{year}/sheets")

            print(f"Doing {BASE_PATH}/{meeting_type}/{year}/diorizations/{dior}")
            with open(
                f"{BASE_PATH}/{meeting_type}/{year}/diorizations/{dior}", "r"
            ) as f:
                current_speaker = ""
                current_speaker_start = 0.0
                for line in f:
                    parts = line.split()
                    start_time = format_seconds(float(parts[3]))
                    duration = float(parts[4])
                    speaker_id = parts[7]
                    if duration < 4:
                        continue
                    if current_speaker != speaker_id:
                        start_times.append(current_speaker_start)
                        durations.append(duration)
                        speaker_ids.append(current_speaker)
                        current_speaker = speaker_id
                        current_speaker_start = start_time

            data = {
                "sprekerID": speaker_ids,
                "StartTijd": start_times,
                "tijd": durations,
            }
            df = pd.DataFrame(data)
            df = df.drop_duplicates()
            df.sort_values(by=["sprekerID", "StartTijd"], inplace=True)
            df.insert(0, "Naam", "")
            df.to_excel(
                f"{BASE_PATH}/{meeting_type}/{year}/sheets/{dior.split()[0]}.xlsx",
                index=False,
            )